# Visibility Tutorial

## Import xradio

In [39]:
import os, pprint
from importlib.metadata import version

try:
    os.system("pip install --upgrade xradio")

    import xradio

    print("Using xradio version", version("xradio"))

except ImportError as exc:
    print(f"Could not import xradio: {exc}")

Using xradio version 0.0.35


## Download example MSv2

## Preparation

In [40]:
import graphviper

graphviper.utils.data.download(file="Antennae_North.cal.lsrk.split.ms")

[2024-08-22 15:57:32,393]  WARNING  graphviper:  File exists: /home/fedemp/ws_xradio_pointing/venv_xradio_python_39/lib/python3.9/site-packages/graphviper/utils/data/.dropbox 
[2024-08-22 15:57:32,394]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List                     
 ────────────────────────────────── 
  Antennae_North.cal.lsrk.split.ms

[2024-08-22 15:57:33,661]     INFO  graphviper:  File exists: Antennae_North.cal.lsrk.split.ms 


## Processing Set

## Convert MSv2 => Processing Set (PS)

In [41]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

msv2_name = "Antennae_North.cal.lsrk.split.ms"
convert_out = "Antennae_North.cal.lsrk.split.vis.zarr"

convert_msv2_to_processing_set(
    in_file=msv2_name,
    out_file=convert_out,
    overwrite=True,
)

[2024-08-22 15:57:33,696]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-22 15:57:33,810]     INFO  graphviper:  Number of partitions: 12 
[2024-08-22 15:57:33,811]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [32 23 30 37], FIELD [0], SCAN [ 9 17 21 25] 
[2024-08-22 15:57:34,006]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [33 24 31], FIELD [1], SCAN [ 9 17 21] 
[2024-08-22 15:57:34,178]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [34 25 32], FIELD [2], SCAN [ 9 17 21] 
[2024-08-22 15:57:34,375]     INFO  graphviper:  OBSERVATION_ID [1], DDI [0], STATE [32 23 30 37], FIELD [0], SCAN [26 34 38 42] 
[2024-08-22 15:57:34,536]     INFO  graphviper:  OBSERVATION_ID [1], DDI [0], STATE [33 24 31], FIELD [1], SCAN [26 34 38] 
[2024-08-22 15:57:34,693]     INFO  graphviper:  OBSERVATION_ID [1], DDI [0], STATE [34 25 32], FIELD [2], SCAN [26 34 38] 
[2024-08-22 15:57:34,851

## Lazy read PS

In [42]:
from xradio.vis.read_processing_set import read_processing_set
convert_out = "Antennae_North.cal.lsrk.split.vis.zarr"

ps = read_processing_set(convert_out, obs_modes=["OBSERVE_TARGET#ON_SOURCE"])

In [43]:
ps.summary()

,name,obs_mode,shape,polarization,spw_name,field_name,source_name,field_coords,start_frequency,end_frequency
0,Antennae_North.cal.lsrk.split_08,OBSERVE_TARGET#ON_SOURCE,"(5, 55, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_2],[NGC4038 - Antennae North_0],"[fk5, 12h01m53.49s, -18d52m02.92s]",3.439281e+11,3.440067e+11
1,Antennae_North.cal.lsrk.split_03,OBSERVE_TARGET#ON_SOURCE,"(20, 55, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_0],[NGC4038 - Antennae North_0],"[fk5, 12h01m52.43s, -18d52m02.92s]",3.439281e+11,3.440067e+11
2,Antennae_North.cal.lsrk.split_01,OBSERVE_TARGET#ON_SOURCE,"(15, 45, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_1],[NGC4038 - Antennae North_0],"[fk5, 12h01m52.96s, -18d52m02.92s]",3.439281e+11,3.440067e+11
3,Antennae_North.cal.lsrk.split_04,OBSERVE_TARGET#ON_SOURCE,"(15, 55, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_1],[NGC4038 - Antennae North_0],"[fk5, 12h01m52.96s, -18d52m02.92s]",3.439281e+11,3.440067e+11
4,Antennae_North.cal.lsrk.split_00,OBSERVE_TARGET#ON_SOURCE,"(20, 45, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_0],[NGC4038 - Antennae North_0],"[fk5, 12h01m52.43s, -18d52m02.92s]",3.439281e+11,3.440067e+11
5,Antennae_North.cal.lsrk.split_06,OBSERVE_TARGET#ON_SOURCE,"(5, 55, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_0],[NGC4038 - Antennae North_0],"[fk5, 12h01m52.43s, -18d52m02.92s]",3.439281e+11,3.440067e+11
6,Antennae_North.cal.lsrk.split_05,OBSERVE_TARGET#ON_SOURCE,"(15, 55, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_2],[NGC4038 - Antennae North_0],"[fk5, 12h01m53.49s, -18d52m02.92s]",3.439281e+11,3.440067e+11
7,Antennae_North.cal.lsrk.split_07,OBSERVE_TARGET#ON_SOURCE,"(5, 55, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_1],[NGC4038 - Antennae North_0],"[fk5, 12h01m52.96s, -18d52m02.92s]",3.439281e+11,3.440067e+11
8,Antennae_North.cal.lsrk.split_02,OBSERVE_TARGET#ON_SOURCE,"(15, 45, 8, 2)","[XX, YY]",spw_0,[NGC4038 - Antennae North_2],[NGC4038 - Antennae North_0],"[fk5, 12h01m53.49s, -18d52m02.92s]",3.439281e+11,3.440067e+11


## PS Structure

A processing set is simply a dictionary of MSv4s (one per observation, field, intent, spectral window - polarization...):

In [44]:
len(ps)

9

In [45]:
ps.keys()

dict_keys(['Antennae_North.cal.lsrk.split_08', 'Antennae_North.cal.lsrk.split_03', 'Antennae_North.cal.lsrk.split_01', 'Antennae_North.cal.lsrk.split_04', 'Antennae_North.cal.lsrk.split_00', 'Antennae_North.cal.lsrk.split_06', 'Antennae_North.cal.lsrk.split_05', 'Antennae_North.cal.lsrk.split_07', 'Antennae_North.cal.lsrk.split_02'])

## MSv4


## Main dataset

We can take one of the items of the Processing Set to look into the contents of that MSv4. Every MSv4 represents the data as an xarray dataset, similarly as in earlier CNGI prototypes. The data variables (visibilities, weights, flags, etc.) can be manipulated and used in computations using the xarray API.

In [46]:
main_xds = ps[
    "Antennae_North.cal.lsrk.split_00"
]

In [47]:
main_xds

<xarray.Dataset> Size: 231kB
Dimensions:                     (time: 20, baseline_id: 45, frequency: 8,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    antenna_id                  (baseline_id) int32 180B dask.array<chunksize=(45,), meta=np.ndarray>
    baseline_antenna1_name      (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
    baseline_antenna2_name      (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 360B 0 1 2 3 ... 41 42 43 44
  * frequency                   (frequency) float64 64B 3.439e+11 ... 3.44e+11
    mount                       (baseline_id) <U6 1kB dask.array<chunksize=(45,), meta=np.ndarray>
    name                        (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
  * polarization                (polarization) <U2 16B 'XX' 'YY'
    station                     (baseline_id) <U4 720B dask.array<chunksize=(45,), meta=np.ndarray>
  * time                        (time) float64 160B 1.307e+09 ... 1.307e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 7kB dask.array<chunksize=(20, 45), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 14kB dask.array<chunksize=(20, 45, 8, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 7kB dask.array<chunksize=(20, 45), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 22kB dask.array<chunksize=(20, 45, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 115kB dask.array<chunksize=(20, 45, 8, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 58kB dask.array<chunksize=(20, 45, 8, 2), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    partition_info:  {'field_name': ['NGC4038 - Antennae North_0'], 'num_line...
    type:            visibility
    weather_xds:     <xarray.Dataset> Size: 27kB\nDimensions:         (statio...
    antenna_xds:     <xarray.Dataset> Size: 2kB\nDimensions:                (...

#### Coordinates

In [48]:
main_xds.polarization

<xarray.DataArray 'polarization' (polarization: 2)> Size: 16B
array(['XX', 'YY'], dtype='<U2')
Coordinates:
  * polarization  (polarization) <U2 16B 'XX' 'YY'

In [49]:
main_xds.uvw_label

<xarray.DataArray 'uvw_label' (uvw_label: 3)> Size: 12B
array(['u', 'v', 'w'], dtype='<U1')
Coordinates:
  * uvw_label  (uvw_label) <U1 12B 'u' 'v' 'w'

In [50]:
main_xds.coords["baseline_id"]

<xarray.DataArray 'baseline_id' (baseline_id: 45)> Size: 360B
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44])
Coordinates:
    antenna_id              (baseline_id) int32 180B dask.array<chunksize=(45,), meta=np.ndarray>
    baseline_antenna1_name  (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
    baseline_antenna2_name  (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
  * baseline_id             (baseline_id) int64 360B 0 1 2 3 4 ... 41 42 43 44
    mount                   (baseline_id) <U6 1kB dask.array<chunksize=(45,), meta=np.ndarray>
    name                    (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
    station                 (baseline_id) <U4 720B dask.array<chunksize=(45,), meta=np.ndarray>

In [51]:
main_xds.time

<xarray.DataArray 'time' (time: 20)> Size: 160B
array([1.306547e+09, 1.306547e+09, 1.306547e+09, 1.306547e+09, 1.306547e+09,
       1.306549e+09, 1.306549e+09, 1.306549e+09, 1.306549e+09, 1.306549e+09,
       1.306550e+09, 1.306550e+09, 1.306550e+09, 1.306550e+09, 1.306550e+09,
       1.306551e+09, 1.306551e+09, 1.306551e+09, 1.306551e+09, 1.306551e+09])
Coordinates:
  * time     (time) float64 160B 1.307e+09 1.307e+09 ... 1.307e+09 1.307e+09
Attributes:
    effective_integration_time:  EFFECTIVE_INTEGRATION_TIME
    format:                      UNIX
    integration_time:            {'attrs': {'type': 'quantity', 'units': ['s'...
    scale:                       utc
    type:                        time
    units:                       ['s']

#### Data vars

In [52]:
main_xds.VISIBILITY

<xarray.DataArray 'VISIBILITY' (time: 20, baseline_id: 45, frequency: 8,
                                polarization: 2)> Size: 115kB
dask.array<open_dataset-VISIBILITY, shape=(20, 45, 8, 2), dtype=complex64, chunksize=(20, 45, 8, 2), chunktype=numpy.ndarray>
Coordinates:
    antenna_id              (baseline_id) int32 180B dask.array<chunksize=(45,), meta=np.ndarray>
    baseline_antenna1_name  (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
    baseline_antenna2_name  (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
  * baseline_id             (baseline_id) int64 360B 0 1 2 3 4 ... 41 42 43 44
  * frequency               (frequency) float64 64B 3.439e+11 ... 3.44e+11
    mount                   (baseline_id) <U6 1kB dask.array<chunksize=(45,), meta=np.ndarray>
    name                    (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
  * polarization            (polarization) <U2 16B 'XX' 'YY'
    station                 (baseline_id) <U4 720B dask.array<chunksize=(45,), meta=np.ndarray>
  * time                    (time) float64 160B 1.307e+09 ... 1.307e+09
Attributes:
    type:                  quanta
    units:                 ['unkown']
    field_and_source_xds:  <xarray.Dataset> Size: 264B\nDimensions:          ...

In [53]:
main_xds.FLAG

<xarray.DataArray 'FLAG' (time: 20, baseline_id: 45, frequency: 8,
                          polarization: 2)> Size: 14kB
dask.array<open_dataset-FLAG, shape=(20, 45, 8, 2), dtype=bool, chunksize=(20, 45, 8, 2), chunktype=numpy.ndarray>
Coordinates:
    antenna_id              (baseline_id) int32 180B dask.array<chunksize=(45,), meta=np.ndarray>
    baseline_antenna1_name  (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
    baseline_antenna2_name  (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
  * baseline_id             (baseline_id) int64 360B 0 1 2 3 4 ... 41 42 43 44
  * frequency               (frequency) float64 64B 3.439e+11 ... 3.44e+11
    mount                   (baseline_id) <U6 1kB dask.array<chunksize=(45,), meta=np.ndarray>
    name                    (baseline_id) <U9 2kB dask.array<chunksize=(45,), meta=np.ndarray>
  * polarization            (polarization) <U2 16B 'XX' 'YY'
    station                 (baseline_id) <U4 720B dask.array<chunksize=(45,), meta=np.ndarray>
  * time                    (time) float64 160B 1.307e+09 ... 1.307e+09

In [54]:
main_xds.VISIBILITY.max()

<xarray.DataArray 'VISIBILITY' ()> Size: 8B
dask.array<_nanmax_skip-aggregate, shape=(), dtype=complex64, chunksize=(), chunktype=numpy.ndarray>

In [55]:
main_xds.VISIBILITY.max().compute()
# main_xds.VISIBILITY.max().values

<xarray.DataArray 'VISIBILITY' ()> Size: 8B
array(6.4229984+0.10052532j, dtype=complex64)

## Metadata

The MS metadata can be found in the attributes of the `main_xds`. Metadata is stored in differente ways:
- in additional xarray sub-datasets, "sub-xds"
- in attributes of coordinates and data variables
- in Python dictionaries.

Most sub-xds are found in the attributes of the `main_xds`, but there are also sub-xds in the attributes of some data variables.
An example of sub-xds of the `main_xds` is the antenna dataset (`antenna_xds`). An example of dictionary is the partition info dict.

### Metadata in sub-xds. Antenna dataset

The MSv4 has xarray datasets in its attributes that represent metadata where n-dimensional arrays is included. Some examples are the `antenna_xds`, `weather_xds` and `pointing_xds`. This would be the equivalent to subtables of the MSv2. Let's look into the antenna sub-xds:


In [56]:
ant_xds = main_xds.attrs["antenna_xds"]

In [57]:
ant_xds

<xarray.Dataset> Size: 2kB
Dimensions:                (name: 10, cartesian_pos_label: 3, receptor_name: 2,
                            sky_dir_label: 2)
Coordinates:
  * cartesian_pos_label    (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                  (name) <U6 240B dask.array<chunksize=(10,), meta=np.ndarray>
  * name                   (name) <U9 360B 'DV02_A015' ... 'PM03_J504'
  * receptor_name          (receptor_name) <U21 168B '0' '1'
  * sky_dir_label          (sky_dir_label) <U3 24B 'ra' 'dec'
    station                (name) <U4 160B dask.array<chunksize=(10,), meta=np.ndarray>
Data variables:
    ANTENNA_DISH_DIAMETER  (name) float64 80B dask.array<chunksize=(10,), meta=np.ndarray>
    ANTENNA_FEED_OFFSET    (name, cartesian_pos_label) float64 240B dask.array<chunksize=(10, 3), meta=np.ndarray>
    ANTENNA_POSITION       (name, cartesian_pos_label) float64 240B dask.array<chunksize=(10, 3), meta=np.ndarray>
    BEAM_OFFSET            (name, receptor_name, sky_dir_label) float64 320B dask.array<chunksize=(10, 2, 2), meta=np.ndarray>
    POLARIZATION_TYPE      (name, receptor_name) <U1 80B dask.array<chunksize=(10, 2), meta=np.ndarray>
    RECEPTOR_ANGLE         (name, receptor_name) float64 160B dask.array<chunksize=(10, 2), meta=np.ndarray>
Attributes:
    overall_telescope_name:  ALMA
    relocatable_antennas:    True

As an xarray dataset, the antenna sub-xds can be used via the same API as the main xds.

In [58]:
ant_xds.ANTENNA_POSITION  # .values to load and see them

<xarray.DataArray 'ANTENNA_POSITION' (name: 10, cartesian_pos_label: 3)> Size: 240B
dask.array<open_dataset-ANTENNA_POSITION, shape=(10, 3), dtype=float64, chunksize=(10, 3), chunktype=numpy.ndarray>
Coordinates:
  * cartesian_pos_label  (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                (name) <U6 240B dask.array<chunksize=(10,), meta=np.ndarray>
  * name                 (name) <U9 360B 'DV02_A015' 'DV06_T704' ... 'PM03_J504'
    station              (name) <U4 160B dask.array<chunksize=(10,), meta=np.ndarray>
Attributes:
    coordinate_system:  geocentric
    ellipsoid:          GRS80
    type:               earth_location
    units:              ['m', 'm', 'm']

In [59]:
ant_xds.name.values

array(['DV02_A015', 'DV06_T704', 'DV07_A004', 'DV08_A072', 'DV09_A008',
       'DV10_A009', 'DV11_A016', 'PM01_T702', 'PM02_A017', 'PM03_J504'],
      dtype='<U9')

### Attributes of Data Arrays and Coordinates. Quantities and Measures

All data variables and coordinates can have quantity and measures information in their attributes section along with other relevant metadata. These measures are specified as dictionaries in the attribute of the data variable or coordinate, with keys `units` and `type` in addition to other keys depending on the type of quantity. The naming conventions are based on `astropy`. For example a quantity of casacore/`position` type, such as the antenna positions, is a quantity with `type: "earth_location"`

For reference, this is the list of measures in the current Processing Set/MSv4 spec:
https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1504318014, with naming conventions based on astropy. For example, a casacore `direction` is a `sky_coord`.



#### Time coordinate
The time coordinate is a time measure (keys: `type`, `units`, `time_scale`, `format`) but also contains for example `integration_time` which is a quantity.

In [60]:
main_xds.time

<xarray.DataArray 'time' (time: 20)> Size: 160B
array([1.306547e+09, 1.306547e+09, 1.306547e+09, 1.306547e+09, 1.306547e+09,
       1.306549e+09, 1.306549e+09, 1.306549e+09, 1.306549e+09, 1.306549e+09,
       1.306550e+09, 1.306550e+09, 1.306550e+09, 1.306550e+09, 1.306550e+09,
       1.306551e+09, 1.306551e+09, 1.306551e+09, 1.306551e+09, 1.306551e+09])
Coordinates:
  * time     (time) float64 160B 1.307e+09 1.307e+09 ... 1.307e+09 1.307e+09
Attributes:
    effective_integration_time:  EFFECTIVE_INTEGRATION_TIME
    format:                      UNIX
    integration_time:            {'attrs': {'type': 'quantity', 'units': ['s'...
    scale:                       utc
    type:                        time
    units:                       ['s']

##### Quantities and measures that are not xarray

When a quantity or a measure is not an xarray, it is specificed as a dictionary with a format based on xarray's [xarray.DataArray.from_dict()](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.from_dict.html) and it has the following keys:
`{"dims": ..., "data": ..., "attrs": quantity/measures_dict}`. The `integration_time` attribute included in the  attributes of the time coordinate is an example:

In [61]:
pprint.pprint(main_xds.time.attrs)

{'effective_integration_time': 'EFFECTIVE_INTEGRATION_TIME',
 'format': 'UNIX',
 'integration_time': {'attrs': {'type': 'quantity', 'units': ['s']},
                      'data': 6.048,
                      'dims': []},
 'scale': 'utc',
 'type': 'time',
 'units': ['s']}


#### Frequency coordinate

The `frequency` coordinate is a `spectral_coord` measure and as such has the following keys in its attributes: `type`, `units`, and `frame`. In addition, the attributes contain the `channel_width`, `spectral_window_name`, and `reference_frequency`.

Any metadata that is a quantity or measure (non-id numbers) is placed in the relevant measures or quantity dictionary.

In [62]:
main_xds.frequency

<xarray.DataArray 'frequency' (frequency: 8)> Size: 64B
array([3.439281e+11, 3.439393e+11, 3.439506e+11, 3.439618e+11, 3.439730e+11,
       3.439843e+11, 3.439955e+11, 3.440067e+11])
Coordinates:
  * frequency  (frequency) float64 64B 3.439e+11 3.439e+11 ... 3.44e+11 3.44e+11
Attributes:
    channel_width:         {'attrs': {'type': 'quantity', 'units': ['Hz']}, '...
    frame:                 LSRK
    reference_frequency:   {'attrs': {'frame': 'LSRK', 'type': 'spectral_coor...
    spectral_window_id:    0
    spectral_window_name:  spw_0
    type:                  spectral_coord
    units:                 ['Hz']

In the frequency coordinate we have example of:
- quantity given as a dict: `channel_width`
- measure given as a dict: `reference_frequency` (a `spectral_coord` ~= casacore/frequency)

In [63]:
pprint.pprint(main_xds.frequency.attrs)

{'channel_width': {'attrs': {'type': 'quantity', 'units': ['Hz']},
                   'data': 11231488.981445312,
                   'dims': []},
 'frame': 'LSRK',
 'reference_frequency': {'attrs': {'frame': 'LSRK',
                                   'type': 'spectral_coord',
                                   'units': ['Hz']},
                         'data': 343928096685.9587,
                         'dims': []},
 'spectral_window_id': 0,
 'spectral_window_name': 'spw_0',
 'type': 'spectral_coord',
 'units': ['Hz']}


### Metadata in dicts. Partition info.

The MSv4 also allows for info dictionaries in the attribute section of the dataset. This is used when no n-dimensional data is required. The relevant measures metadata is included, similarly as with coordinates and data variables (when non-id) in xarray datasets.

An example is the `partition_info` dict.

An example is the field_info where the delay_direction, phase_direction, and reference_direction are stored as `sky_coord` measures (keys: `type`, `units`, `reference_frame`).

In [64]:
main_xds.partition_info

{'field_name': ['NGC4038 - Antennae North_0'],
 'num_lines': 0,
 'obs_mode': 'OBSERVE_TARGET#ON_SOURCE',
 'polarization_setup': ['XX', 'YY'],
 'source_name': ['NGC4038 - Antennae North_0'],
 'spectral_window_name': 'spw_0',
 'taql': 'WHERE (DATA_DESC_ID IN [0]) AND(OBSERVATION_ID IN [0]) AND(STATE_ID IN [32,23,30,37]) AND(FIELD_ID IN [0]) AND(SCAN_NUMBER IN [9,17,21,25]) AND(STATE_ID IN [32,23,30,37]) '}

### Metadata in sub-xds of data variables. Field_and_source sub-dataset.

A special example of sub-xds is the `xds` which is included in the attributes of the VISIBILITY data variable. This way, transformations applied on the visibilities can be reflected in variables such as the field phase center or the source direction. Here data variables such as `FIELD_PHASE_CENTER` or `SOURCE_DIRECTION` are stored as `sky_coord` measures (their attributes contain the following keys: `type`, `units`, `frame`).

In [65]:
field_and_source_xds = main_xds.VISIBILITY.field_and_source_xds

In [66]:
field_and_source_xds

<xarray.Dataset> Size: 264B
Dimensions:             (sky_dir_label: 2)
Coordinates:
    field_name          <U26 104B ...
  * sky_dir_label       (sky_dir_label) <U3 24B 'ra' 'dec'
    source_name         <U26 104B ...
Data variables:
    FIELD_PHASE_CENTER  (sky_dir_label) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
    SOURCE_LOCATION     (sky_dir_label) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
Attributes:
    is_ephemeris:  False

In [67]:
field_and_source_xds.FIELD_PHASE_CENTER

<xarray.DataArray 'FIELD_PHASE_CENTER' (sky_dir_label: 2)> Size: 16B
dask.array<open_dataset-FIELD_PHASE_CENTER, shape=(2,), dtype=float64, chunksize=(2,), chunktype=numpy.ndarray>
Coordinates:
    field_name     <U26 104B ...
  * sky_dir_label  (sky_dir_label) <U3 24B 'ra' 'dec'
    source_name    <U26 104B ...
Attributes:
    frame:    fk5
    type:     sky_coord
    units:    ['rad', 'rad']

In [68]:
field_and_source_xds.SOURCE_LOCATION

<xarray.DataArray 'SOURCE_LOCATION' (sky_dir_label: 2)> Size: 16B
dask.array<open_dataset-SOURCE_LOCATION, shape=(2,), dtype=float64, chunksize=(2,), chunktype=numpy.ndarray>
Coordinates:
    field_name     <U26 104B ...
  * sky_dir_label  (sky_dir_label) <U3 24B 'ra' 'dec'
    source_name    <U26 104B ...
Attributes:
    frame:    fk5
    type:     sky_coord
    units:    ['rad', 'rad']

## Selection examples

One can use the usual selection functionality of xarray with all arrays, the main dataset and all sub datasets. For example, selection by labels, `sel()`:

In [69]:
sel_xds = main_xds.sel(frequency=slice(3.43939e11, 3.4397e11))
sel_xds.frequency

<xarray.DataArray 'frequency' (frequency: 3)> Size: 24B
array([3.439393e+11, 3.439506e+11, 3.439618e+11])
Coordinates:
  * frequency  (frequency) float64 24B 3.439e+11 3.44e+11 3.44e+11
Attributes:
    channel_width:         {'attrs': {'type': 'quantity', 'units': ['Hz']}, '...
    frame:                 LSRK
    reference_frequency:   {'attrs': {'frame': 'LSRK', 'type': 'spectral_coor...
    spectral_window_id:    0
    spectral_window_name:  spw_0
    type:                  spectral_coord
    units:                 ['Hz']

Or selection by indices, `isel()`

In [70]:
isel_xds = main_xds.isel(frequency=slice(1, 4))
isel_xds.frequency

<xarray.DataArray 'frequency' (frequency: 3)> Size: 24B
array([3.439393e+11, 3.439506e+11, 3.439618e+11])
Coordinates:
  * frequency  (frequency) float64 24B 3.439e+11 3.44e+11 3.44e+11
Attributes:
    channel_width:         {'attrs': {'type': 'quantity', 'units': ['Hz']}, '...
    frame:                 LSRK
    reference_frequency:   {'attrs': {'frame': 'LSRK', 'type': 'spectral_coor...
    spectral_window_id:    0
    spectral_window_name:  spw_0
    type:                  spectral_coord
    units:                 ['Hz']

In [71]:
sel_xds.equals(isel_xds)

True

In [72]:
sel_xds.identical(isel_xds)

True